In [351]:
base=10

In [352]:
def count(func):
    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

In [353]:
import time
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        # print(f"Took {end - start} seconds")
        return end - start #result
    return wrapper

In [354]:
@count
def add_mac(a, b) :
    c = a + b
    return c%base, c//base

In [355]:
def add1(alpha, beta, c) :
    r = add_mac(alpha[0], beta[0])
    s = add_mac(r[0], c)
    t = add_mac(r[1], s[1])
    # assert t[1] == 0  # comment out
    if len(beta) == 1 : return [s[0], t[0]]
    return [s[0]] + add1(alpha[1:], beta[1:], t[0])

In [356]:
def add(alpha, beta) :
    alpha, beta = alpha[::-1], beta[::-1]
    la, lb = len(alpha), len(beta)
    l = max(la, lb)
    alpha = alpha + (l-la) * [0]
    beta = beta + (l-lb) * [0]
    result = add1(alpha, beta, 0)[::-1]
    if result[0] == 0 : return result[1:]
    return result

In [377]:
def subtract(alpha, beta) :
    print('\n!')
    beta = [0] * (len(alpha) - len(beta)) + beta
    if alpha == beta : return [0]
    print(f'{alpha=}, {beta=}')
    beta = [base-1-b for b in beta]
    print(f'{beta=}')
    result = add(alpha, beta)
    print(f'{result=}')
    if result[0] == 1 : return add(result[1:], [1])
    raise ValueError("Negative result")

In [378]:
def parse(input, base=base) :
    arr = []
    while input >= base :
        arr.append(input%base)
        input = input//base
    arr.append(input)
    return arr[::-1]

In [379]:
def pad(num1, num2) :
    max_len = max(len(num1), len(num2))
    pad_len = 2 ** (len(bin(max_len-1)[2:]))
    # pad_len = max_len + 1
    num1 = [0] * (pad_len - len(num1)) + num1
    num2 = [0] * (pad_len - len(num2)) + num2
    return num1, num2

In [397]:
@count
def karatsuba(num1, num2) :

    len1 = len(num1)
    len2 = len(num2)
    if len1 == 0 or len2 == 0 : return [0]
    length = max(len1, len2)
    print(num1, num2, len(num1), len(num2), length)
    
    if length == 1 : return parse(num1[0] * num2[0])
    half_len = length//2
    af, al = num1[:half_len], num1[half_len:]
    bf, bl = num2[:half_len], num2[half_len:]

    p = karatsuba(al, bl)
    q = karatsuba(add(af, al), add(bf, bl))
    r = karatsuba(af, bf)

    print(f'{p=}, {q=}, {r=}')
    print(f'{add(af, al)=}, {add(bf, bl)=}')

    m = add(add(r+[0]*length, q+[0]*half_len), p)
    return subtract(m, add(p, r)+[0]*half_len)


In [398]:
@timer
def karatsuba_unpadded(num1, num2) :
    num1, num2 = pad(parse(num1), parse(num2))
    print(num1, num2)
    return karatsuba(num1, num2)  

In [399]:
karatsuba_unpadded(10, 10)

[0, 0, 1, 0] [0, 0, 1, 0]
[0, 0, 1, 0] [0, 0, 1, 0] 4 4 4
[1, 0] [1, 0] 2 2 2
[0] [0] 1 1 1
[1] [1] 1 1 1
[1] [1] 1 1 1
p=[0], q=[1], r=[1]
add(af, al)=[1], add(bf, bl)=[1]

!
alpha=[1, 1, 0], beta=[0, 1, 0]
beta=[9, 8, 9]
result=[1, 0, 9, 9]
[1, 0] [1, 0] 2 2 2
[0] [0] 1 1 1
[1] [1] 1 1 1
[1] [1] 1 1 1
p=[0], q=[1], r=[1]
add(af, al)=[1], add(bf, bl)=[1]

!
alpha=[1, 1, 0], beta=[0, 1, 0]
beta=[9, 8, 9]
result=[1, 0, 9, 9]
[0, 0] [0, 0] 2 2 2
[0] [0] 1 1 1
[0] [0] 1 1 1
[0] [0] 1 1 1
p=[0], q=[0], r=[0]
add(af, al)=[0], add(bf, bl)=[0]

!
p=[1, 0, 0], q=[1, 0, 0], r=[0]
add(af, al)=[1, 0], add(bf, bl)=[1, 0]

!
alpha=[1, 0, 1, 0, 0], beta=[1, 0, 0, 0, 0]
beta=[8, 9, 9, 9, 9]
result=[1, 0, 0, 0, 9, 9]


0.0005161762237548828

In [400]:
add_mac.calls

282

In [401]:
from random import uniform
input_lengths = range(1, 11)
num_adds = []
num_karatsubas = []
karatsuba_times = []

print('input_len \t add_mac calls \t karatsuba calls \t time (s)')

for input_length in input_lengths :
    add_mac.calls, karatsuba.calls = 0, 0
    num1 = int(uniform(10**(input_length-1), 10**input_length))
    num2 = int(uniform(10**(input_length-1), 10**input_length))
    print(f'{num1=}, {num2=}')
    time_taken = karatsuba_unpadded(num1, num2)
    karatsuba_times.append(time_taken)
    num_adds.append(add_mac.calls)
    num_karatsubas.append(karatsuba.calls)
    print(f'{input_length:02d} \t\t {add_mac.calls:04d} \t\t {karatsuba.calls:04d} \t\t {time_taken:.2e}')

input_len 	 add_mac calls 	 karatsuba calls 	 time (s)
num1=7, num2=3
[0, 7] [0, 3]
[0, 7] [0, 3] 2 2 2
[7] [3] 1 1 1
[7] [3] 1 1 1
[0] [0] 1 1 1
p=[2, 1], q=[2, 1], r=[0]
add(af, al)=[7], add(bf, bl)=[3]

!
alpha=[2, 3, 1], beta=[2, 1, 0]
beta=[7, 8, 9]
result=[1, 0, 2, 0]
01 		 0054 		 0004 		 1.23e-04
num1=73, num2=67
[0, 0, 7, 3] [0, 0, 6, 7]
[0, 0, 7, 3] [0, 0, 6, 7] 4 4 4
[7, 3] [6, 7] 2 2 2
[3] [7] 1 1 1
[1, 0] [1, 3] 2 2 2
[0] [3] 1 1 1
[1] [4] 1 1 1
[1] [1] 1 1 1
p=[0], q=[4], r=[1]
add(af, al)=[1], add(bf, bl)=[4]

!
alpha=[1, 4, 0], beta=[0, 1, 0]
beta=[9, 8, 9]
result=[1, 1, 2, 9]
[7] [6] 1 1 1
p=[2, 1], q=[1, 3, 0], r=[4, 2]
add(af, al)=[1, 0], add(bf, bl)=[1, 3]

!
alpha=[5, 5, 2, 1], beta=[0, 6, 3, 0]
beta=[9, 3, 6, 9]
result=[1, 4, 8, 9, 0]
[7, 3] [6, 7] 2 2 2
[3] [7] 1 1 1
[1, 0] [1, 3] 2 2 2
[0] [3] 1 1 1
[1] [4] 1 1 1
[1] [1] 1 1 1
p=[0], q=[4], r=[1]
add(af, al)=[1], add(bf, bl)=[4]

!
alpha=[1, 4, 0], beta=[0, 1, 0]
beta=[9, 8, 9]
result=[1, 1, 2, 9]
[7] [6] 1 1 1


ValueError: Negative result